# Breast Cancer Segmentation with MSGRAP

We will designing and developing MSGRAP DL segmentation model based on H. Lee's research paper. We will adapt it to be trained on breast cancer MRI dataset since H. Lee originally trained it on breast cancer ultrasound images. We will note down the differences as we work out the implementation.

- Dynamic contrast-enhanced magnetic resonance images of breast cancer patients with tumor locations - Imaging Cancer Archive - **Duke-Breast-Cancer-MRI Dataset**: https://wiki.cancerimagingarchive.net/pages/viewpage.action?pageId=70226903


[1] H. Lee, J. Park and J. Y. Hwang, "Channel Attention Module With Multiscale Grid Average Pooling for Breast Cancer Segmentation in an Ultrasound Image," in IEEE Transactions on Ultrasonics, Ferroelectrics, and Frequency Control, vol. 67, no. 7, pp. 1344-1353, July 2020, doi: 10.1109/TUFFC.2020.2972573

Referenced 2D Channel - Spatial Attention (Squeeze & Excite) VGG based TensorFlow Keras code in tutorial Attending to Channels Using Keras and TensorFlow

The segmentation mask files in this supplemental folder have the following naming scheme:

- `Breast_MRI_{patient_ID}_{segmentation_tissue_target}.seg.nrrd`

~~~bash
# Folder Structure

.
├── Segmentation_Masks_NRRD       		           					# Directory containing true segmentation masks
│   ├── Breast_MRI_002       										# Subject directory containing breast, FGT, and blood vessel mask
│   |   ├── Segmentation_Breast_MRI_002_Breast.seg.nrrd 			# Breast mask for subject Breast_MRI_002
│   |   ├── Segmentation_Breast_MRI_002_Dense_and_Vessels.seg.nrrd  # FGT and blood vessel mask for subject Breast_MRI_002
│   ...
~~~


## Outline¶

- Prepare Tcia Breast Diagnosis Data
- Breast Segmentation Model Architecture
- Train Breast Segmentation ML/DL Models
- Evaluate Breast Segmentation ML/DL Models Quantitatively
- Evaluate Breast Segmentation ML/DL Models Qualitatively
- Deploy Breast Segmentation DL Model for Inference

In [1]:
import tensorflow as tf
# tf.compat.v1.enable_eager_execution()
from tensorflow import keras
from tensorflow.keras.layers import *
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import UpSampling2D
from tensorflow.keras.layers import MaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.layers import concatenate,Dropout
from tensorflow.keras.layers import Multiply, MaxPooling2D, GlobalMaxPooling2D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Input, Add, Dense, Activation, ZeroPadding2D
from tensorflow.keras.layers import BatchNormalization, Flatten, Conv2D, AveragePooling2D
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.utils import plot_model
from tensorflow.keras.initializers import glorot_uniform
from tensorflow.keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
import pandas as pd
import numpy as np
import os
import pydicom as dicom
from tqdm import tqdm
from skimage.io import imsave
import matplotlib.pylab as plt
from random import choice
from glob import glob

## Prepare Duke's Breast Cancer MRI Seg Data

### Load Breast Cancer MRI Metadata CSV

In [3]:
duke_bc_base_dir = os.path.join("/media","james", "My Passport",
    "Jetson_TX2_CMPE258", "duke-breast-cancer-mri")

# data_path is duke_bc_data_dir
duke_bc_data_dir = duke_bc_base_dir + \
    os.path.join("/Duke-Breast-Cancer-MRI_v2_20220609", 
                 "manifest-1654812109500")
seg_file_mapping = "segmentation_filepath_mapping.csv"

# data path to breast cancer, FGT/Dense and tissue segmentation masks
duke_bc_seg_data_dir = os.path.join(duke_bc_base_dir, 
    "Duke-Breast-Cancer-MRI-Supplement-v3", "Segmentation_Masks_NRRD")

In [4]:
bc_mri_metadata = pd.read_csv(duke_bc_data_dir + "/metadata.csv")
bc_mri_metadata.head(5)

,Series UID,Collection,3rd Party Analysis,Data Description URI,Subject ID,Study UID,Study Description,Study Date,Series Description,Manufacturer,Modality,SOP Class Name,SOP Class UID,Number of Images,File Size,File Location,Download Timestamp
0,1.3.6.1.4.1.14519.5.2.1.1754149663016455182384...,Duke-Breast-Cancer-MRI,NO,https://doi.org/10.7937/TCIA.e3svre93,Breast_MRI_001,1.3.6.1.4.1.14519.5.2.1.1860515210678639712695...,MRI BREAST BILATERAL WWO,01-01-1990,ax t1 tse c,SIEMENS,MR,MR Image Storage,1.2.840.10008.5.1.4.1.1.4,42,5.60 MB,./Duke-Breast-Cancer-MRI/Breast_MRI_001/01-01-...,2022-11-20T08:47:56.032
1,1.3.6.1.4.1.14519.5.2.1.1857778498036652445367...,Duke-Breast-Cancer-MRI,NO,https://doi.org/10.7937/TCIA.e3svre93,Breast_MRI_001,1.3.6.1.4.1.14519.5.2.1.1860515210678639712695...,MRI BREAST BILATERAL WWO,01-01-1990,ax dyn pre,SIEMENS,MR,MR Image Storage,1.2.840.10008.5.1.4.1.1.4,160,64.60 MB,./Duke-Breast-Cancer-MRI/Breast_MRI_001/01-01-...,2022-11-20T08:48:51.95
2,1.3.6.1.4.1.14519.5.2.1.1072629657499780353316...,Duke-Breast-Cancer-MRI,NO,https://doi.org/10.7937/TCIA.e3svre93,Breast_MRI_001,1.3.6.1.4.1.14519.5.2.1.1860515210678639712695...,MRI BREAST BILATERAL WWO,01-01-1990,ax dyn 3rd pass,SIEMENS,MR,MR Image Storage,1.2.840.10008.5.1.4.1.1.4,160,64.61 MB,./Duke-Breast-Cancer-MRI/Breast_MRI_001/01-01-...,2022-11-20T08:48:54.638
3,1.3.6.1.4.1.14519.5.2.1.1514720349923003958578...,Duke-Breast-Cancer-MRI,NO,https://doi.org/10.7937/TCIA.e3svre93,Breast_MRI_001,1.3.6.1.4.1.14519.5.2.1.1860515210678639712695...,MRI BREAST BILATERAL WWO,01-01-1990,ax dyn 1st pass,SIEMENS,MR,MR Image Storage,1.2.840.10008.5.1.4.1.1.4,160,64.61 MB,./Duke-Breast-Cancer-MRI/Breast_MRI_001/01-01-...,2022-11-20T08:48:56.832
4,1.3.6.1.4.1.14519.5.2.1.1602809643137194123479...,Duke-Breast-Cancer-MRI,NO,https://doi.org/10.7937/TCIA.e3svre93,Breast_MRI_002,1.3.6.1.4.1.14519.5.2.1.2934485107964891261049...,MRI BREAST BILATERAL W WO,01-01-1990,ax t1,GE MEDICAL SYSTEMS,MR,MR Image Storage,1.2.840.10008.5.1.4.1.1.4,60,31.59 MB,./Duke-Breast-Cancer-MRI/Breast_MRI_002/01-01-...,2022-11-20T08:49:37.536


### Load Breast Cancer Segmentation Filepath Mapping CSV

In [5]:
bc_seg_file_mapping_df = pd.read_csv(duke_bc_base_dir + os.sep + seg_file_mapping)
bc_seg_file_mapping_df.head()

,Patient ID,Segmentation Label,Slice File,Full Descriptive Path
0,Breast_MRI_002,Fatty tissue of breast,1-125.dcm,Duke-Breast-Cancer-MRI/Breast_MRI_002/01-01-19...
1,Breast_MRI_002,Fatty tissue of breast,1-030.dcm,Duke-Breast-Cancer-MRI/Breast_MRI_002/01-01-19...
2,Breast_MRI_002,Fatty tissue of breast,1-073.dcm,Duke-Breast-Cancer-MRI/Breast_MRI_002/01-01-19...
3,Breast_MRI_002,Mammary Fibroglandular Tissue,1-116.dcm,Duke-Breast-Cancer-MRI/Breast_MRI_002/01-01-19...
4,Breast_MRI_002,Mammary Fibroglandular Tissue,1-073.dcm,Duke-Breast-Cancer-MRI/Breast_MRI_002/01-01-19...


### Load Segmentation Breast & FGT/Blood Vessel Masks Train

In [6]:
patients = sorted(os.listdir(duke_bc_seg_data_dir))

In [11]:
print("Num Patients: {}".format(len(patients)))
patients[:5]

Num Patients: 100


['Breast_MRI_002',
 'Breast_MRI_006',
 'Breast_MRI_018',
 'Breast_MRI_021',
 'Breast_MRI_023']

In [ ]:
## NOTE: Train & Test Split can be done here by patient ids 80/20

In [8]:
train_bc_mask_paths = [sorted(glob(
    duke_bc_seg_data_dir + os.sep + patient_id + "/*seg.nrrd")
                             ) for patient_id in patients]

In [9]:
train_bc_mask_paths[:5]

[['/media/james/My Passport/Jetson_TX2_CMPE258/duke-breast-cancer-mri/Duke-Breast-Cancer-MRI-Supplement-v3/Segmentation_Masks_NRRD/Breast_MRI_002/Segmentation_Breast_MRI_002_Breast.seg.nrrd',
  '/media/james/My Passport/Jetson_TX2_CMPE258/duke-breast-cancer-mri/Duke-Breast-Cancer-MRI-Supplement-v3/Segmentation_Masks_NRRD/Breast_MRI_002/Segmentation_Breast_MRI_002_Dense_and_Vessels.seg.nrrd'],
 ['/media/james/My Passport/Jetson_TX2_CMPE258/duke-breast-cancer-mri/Duke-Breast-Cancer-MRI-Supplement-v3/Segmentation_Masks_NRRD/Breast_MRI_006/Segmentation_Breast_MRI_006_Breast.seg.nrrd',
  '/media/james/My Passport/Jetson_TX2_CMPE258/duke-breast-cancer-mri/Duke-Breast-Cancer-MRI-Supplement-v3/Segmentation_Masks_NRRD/Breast_MRI_006/Segmentation_Breast_MRI_006_Dense_and_Vessels.seg.nrrd'],
 ['/media/james/My Passport/Jetson_TX2_CMPE258/duke-breast-cancer-mri/Duke-Breast-Cancer-MRI-Supplement-v3/Segmentation_Masks_NRRD/Breast_MRI_018/Segmentation_Breast_MRI_018_Breast.seg.nrrd',
  '/media/james/